Processamento de dados:
- criar tabela com todos os MQLs, com todas as informações disponíveis para cada um.
- criar uma coluna com quais MQLs fecharam acordo e quais foram perdidos.

Perguntas a serem respondidas:
- Qual foi a taxa de conversão total?
- Qual foi a taxa de conversão de cada origem?
- Qual foi a taxa de conversão de cada página inicial?
- Para cada SR, qual a receita média declarada dos leads?
- Para cada SR, quantos de cada business_type a pessoa converteu?
- Para cada SR, quantos de cada lead_behaviour_profile a pessoa converteu?
- Para cada SR, quantos de cada lead_type a pessoa converteu?

Extra:

Caso consiga resolver as questões acima com facilidade, aproveite a base "irmã" dessa para fazer outras explorações que você ache interessante!

https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

Lembre de separar novamente em 2 etapas: 
1. Processamento dos dados, para criar uma tabela "analítica".
2. Análise exploratória dos dados.

In [86]:
import numpy as np
import pandas as pd

In [2]:
df_marketing = pd.read_csv('dados/olist_marketing_qualified_leads_dataset.csv', parse_dates=['first_contact_date'])
df_marketing.head()

,mql_id,first_contact_date,landing_page_id,origin
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search


In [3]:
df_marketing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   mql_id              8000 non-null   object        
 1   first_contact_date  8000 non-null   datetime64[ns]
 2   landing_page_id     8000 non-null   object        
 3   origin              7940 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 250.1+ KB


In [4]:
df_deals = pd.read_csv('dados/olist_closed_deals_dataset.csv', parse_dates=['won_date'])
df_deals.head()

,mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,5420aad7fec3549a85876ba1c529bd84,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0
1,a555fb36b9368110ede0f043dfc3b9a0,bbb7d7893a450660432ea6652310ebb7,09285259593c61296eef10c734121d5b,d3d1e91a157ea7f90548eef82f1955e3,2018-05-08 20:17:59,car_accessories,industry,eagle,NaN,NaN,NaN,reseller,NaN,0.0
2,327174d3648a2d047e8940d7d15204ca,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,cat,NaN,NaN,NaN,reseller,NaN,0.0
3,f5fee8f7da74f4887f5bcae2bafb6dd6,21e1781e36faf92725dde4730a88ca0f,56bf83c4bb35763a51c2baab501b4c67,d3d1e91a157ea7f90548eef82f1955e3,2018-01-17 13:51:03,food_drink,online_small,NaN,NaN,NaN,NaN,reseller,NaN,0.0
4,ffe640179b554e295c167a2f6be528e0,ed8cb7b190ceb6067227478e48cf8dde,4b339f9567d060bcea4f5136b9f5949e,d3d1e91a157ea7f90548eef82f1955e3,2018-07-03 20:17:45,home_appliances,industry,wolf,NaN,NaN,NaN,manufacturer,NaN,0.0


In [5]:
df_deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   mql_id                         842 non-null    object        
 1   seller_id                      842 non-null    object        
 2   sdr_id                         842 non-null    object        
 3   sr_id                          842 non-null    object        
 4   won_date                       842 non-null    datetime64[ns]
 5   business_segment               841 non-null    object        
 6   lead_type                      836 non-null    object        
 7   lead_behaviour_profile         665 non-null    object        
 8   has_company                    63 non-null     object        
 9   has_gtin                       64 non-null     object        
 10  average_stock                  66 non-null     object        
 11  business_type      

In [14]:
df_mql_total = pd.merge(df_marketing, df_deals, on='mql_id', how='left')
df_mql_total.head()

,mql_id,first_contact_date,landing_page_id,origin,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0


In [15]:
df_mql_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 0 to 7999
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   mql_id                         8000 non-null   object        
 1   first_contact_date             8000 non-null   datetime64[ns]
 2   landing_page_id                8000 non-null   object        
 3   origin                         7940 non-null   object        
 4   seller_id                      842 non-null    object        
 5   sdr_id                         842 non-null    object        
 6   sr_id                          842 non-null    object        
 7   won_date                       842 non-null    datetime64[ns]
 8   business_segment               841 non-null    object        
 9   lead_type                      836 non-null    object        
 10  lead_behaviour_profile         665 non-null    object        
 11  has_company      

In [28]:
df_mql_total['closed_deal'] = ~df_mql_total['sr_id'].isnull()
df_mql_total.head()

,mql_id,first_contact_date,landing_page_id,origin,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue,closed_deal
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0,True


### Qual foi a taxa de conversão total?

In [38]:
qtde_vendas = len(df_mql_total[df_mql_total['closed_deal'] == True])
qtde_total = len(df_mql_total)
tx_conversao = (qtde_vendas/qtde_total)*100
print("A taxa de conversão total foi de: ", tx_conversao)

A taxa de conversão total foi de:  10.525


### Qual foi a taxa de conversão de cada origem?

In [56]:
df_origin_true = df_mql_total[df_mql_total['closed_deal'] == True].groupby('origin').agg({'closed_deal':'count'})
df_origin_total = df_mql_total.groupby('origin').agg({'closed_deal':'count'})
df_tx_conversao_origin = ((df_origin_true/df_origin_total)*100)
df_tx_conversao_origin

,closed_deal
origin,
direct_traffic,11.222445
display,5.084746
email,3.042596
organic_search,11.803136
other,2.666667
other_publicities,4.615385
paid_search,12.295082
referral,8.450704
social,5.555556


### Qual foi a taxa de conversão de cada página inicial?

In [63]:
df_landing_true = df_mql_total[df_mql_total['closed_deal'] == True].groupby('landing_page_id').agg({'closed_deal':'count'})
df_landing_total = df_mql_total.groupby('landing_page_id').agg({'closed_deal':'count'})
df_tx_conversao_landing = ((df_landing_true/df_landing_total)*100).round(2)
df_tx_conversao_landing

,closed_deal
landing_page_id,
007f9098284a86ee80ddeb25d53e0af8,6.06
01850876d34b43494b5f9879c366877b,NaN
0218f6be0b76aca72ab4d00ee9e8cf10,12.50
03c7c0ace395d80182db07ae2c30f034,NaN
04a75b75d5f4f3a16b0178fbe382da00,100.00
...,...
fd7c4cdfbfb7964b441618d54f88c38b,NaN
fd7d977e75d1ac5cfe6b93ffabb150b1,NaN
fe82aeadeaea2186478fe6353d6d5301,25.00


In [64]:
df_tx_conversao_landing = df_tx_conversao_landing.fillna(0)
df_tx_conversao_landing

,closed_deal
landing_page_id,
007f9098284a86ee80ddeb25d53e0af8,6.06
01850876d34b43494b5f9879c366877b,0.00
0218f6be0b76aca72ab4d00ee9e8cf10,12.50
03c7c0ace395d80182db07ae2c30f034,0.00
04a75b75d5f4f3a16b0178fbe382da00,100.00
...,...
fd7c4cdfbfb7964b441618d54f88c38b,0.00
fd7d977e75d1ac5cfe6b93ffabb150b1,0.00
fe82aeadeaea2186478fe6353d6d5301,25.00


### Para cada SR, qual a receita média declarada dos leads?

In [85]:
df_revenue = df_mql_total.groupby('sr_id').agg({'declared_monthly_revenue':'mean'}).round(2)
df_revenue

,declared_monthly_revenue
sr_id,
060c0a26f19f4d66b42e0d8796688490,0.00
068066e24f0c643eb1d089c7dd20cd73,7037.04
0a0fb2b07d841f84fb6714e35c723075,6000.00
2695de1affa7750089c0455f8ce27021,1694.92
34d40cdaf94010a1d05b0d6212f9e909,19000.00
495d4e95a8cf8bbf8b432b612a2aa328,6666.67
4b339f9567d060bcea4f5136b9f5949e,27777.78
4ef15afb4b2723d8f3d81e51ec7afefe,377293.23
56bf83c4bb35763a51c2baab501b4c67,12500.00


### Para cada SR, quantos de cada business_type a pessoa converteu?

In [73]:
df_mql_total['business_type'].unique()

array([nan, 'reseller', 'manufacturer', 'other'], dtype=object)

In [84]:
df_business_type = df_mql_total[df_mql_total['closed_deal'] == True].groupby(['sr_id', 'business_type']).agg({'closed_deal':'count'})
df_business_type

closed_deal
sr_id                            business_type             
060c0a26f19f4d66b42e0d8796688490 manufacturer             9
                                 reseller                23
068066e24f0c643eb1d089c7dd20cd73 manufacturer             6
                                 reseller                21
0a0fb2b07d841f84fb6714e35c723075 other                    1
2695de1affa7750089c0455f8ce27021 manufacturer            16
                                 reseller                42
34d40cdaf94010a1d05b0d6212f9e909 manufacturer             1
                                 other                    1
                                 reseller                 8
495d4e95a8cf8bbf8b432b612a2aa328 manufacturer            14
                                 reseller                47
4b339f9567d060bcea4f5136b9f5949e manufacturer             2
                                 reseller                 6
4ef15afb4b2723d8f3d81e51ec7afefe manufacturer            43
                                 reseller                90
56bf83c4bb35763a51c2baab501b4c67 manufacturer             8
                                 reseller                16
6565aa9ce3178a5caf6171827af3a9ba manufacturer            19
                                 reseller                54
6aa3b86a83d784b05f0e37e26b20860d reseller                 1
85fc447d336637ba1df43e793199fbc8 manufacturer            25
                                 reseller                39
9749123c950bf8363ace42cb1c2d0815 manufacturer             2
                                 reseller                 5
9ae085775a198122c5586fa830ff7f2b manufacturer            15
                                 reseller                36
9d12ef1a7eca3ec58c545c678af7869c manufacturer             2
                                 reseller                 4
9e4d1098a3b0f5da39b0bc48f9876645 manufacturer            10
                                 reseller                14
a8387c01a09e99ce014107505b92388c manufacturer             4
                                 other                    1
                                 reseller                21
b90f87164b5f8c2cfa5c8572834dbe3f reseller                 1
c638112b43f1d1b86dcabb0da720c901 manufacturer            12
                                 reseller                22
d3d1e91a157ea7f90548eef82f1955e3 manufacturer            19
                                 reseller                61
de63de0d10a6012430098db33c679b0b manufacturer            12
                                 reseller                40
fbf4aef3f6915dc0c3c97d6812522f6a manufacturer            23
                                 reseller                36

### Para cada SR, quantos de cada lead_behaviour_profile a pessoa converteu?

In [76]:
df_mql_total['lead_behaviour_profile'].unique()

array([nan, 'cat', 'eagle', 'wolf', 'shark', 'cat, wolf', 'eagle, wolf',
       'shark, cat', 'eagle, cat', 'shark, wolf'], dtype=object)

In [83]:
df_lead_behaviour = df_mql_total[df_mql_total['closed_deal'] == True].groupby(['sr_id', 'lead_behaviour_profile']).agg({'closed_deal':'count'})
df_lead_behaviour

closed_deal
sr_id                            lead_behaviour_profile             
060c0a26f19f4d66b42e0d8796688490 cat                              15
                                 eagle                             2
                                 shark                             1
                                 wolf                              6
068066e24f0c643eb1d089c7dd20cd73 cat                              11
...                                                              ...
de63de0d10a6012430098db33c679b0b wolf                              5
fbf4aef3f6915dc0c3c97d6812522f6a cat                              30
                                 eagle                             9
                                 shark                             2
                                 wolf                              2

[80 rows x 1 columns]

### Para cada SR, quantos de cada lead_type a pessoa converteu?

In [80]:
df_mql_total['lead_type'].unique()

array([nan, 'online_medium', 'industry', 'online_big', 'online_small',
       'offline', 'online_top', 'online_beginner', 'other'], dtype=object)

In [82]:
df_lead_type = df_mql_total[df_mql_total['closed_deal'] == True].groupby(['sr_id', 'lead_type']).agg({'closed_deal':'count'})
df_lead_type

closed_deal
sr_id                            lead_type                   
060c0a26f19f4d66b42e0d8796688490 industry                   8
                                 offline                    1
                                 online_beginner            3
                                 online_big                 2
                                 online_medium             15
...                                                       ...
fbf4aef3f6915dc0c3c97d6812522f6a online_beginner            1
                                 online_big                 8
                                 online_medium             30
                                 online_small               6
                                 online_top                 2

[118 rows x 1 columns]